# Spark Preparation
We check if we are in Google Colab.  If this is the case, install all necessary packages.

To run spark in Colab, we need to first install all the dependencies in Colab environment i.e. Apache Spark 3.2.1 with hadoop 3.2, Java 8 and Findspark to locate the spark in the system. The tools installation can be carried out inside the Jupyter Notebook of the Colab.
Learn more from [A Must-Read Guide on How to Work with PySpark on Google Colab for Data Scientists!](https://www.analyticsvidhya.com/blog/2020/11/a-must-read-guide-on-how-to-work-with-pyspark-on-google-colab-for-data-scientists/)

In [ ]:
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

In [ ]:
if IN_COLAB:
    !apt-get install openjdk-8-jdk-headless -qq > /dev/null
    !wget -q https://dlcdn.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
    !tar xf spark-3.2.1-bin-hadoop3.2.tgz
    !mv spark-3.2.1-bin-hadoop3.2 spark
    !pip install -q findspark

In [ ]:
if IN_COLAB:
  import os
  os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
  os.environ["SPARK_HOME"] = "/content/spark"

# Start a Local Cluster
Use findspark.init() to start a local cluster.  If you plan to use remote cluster, skip the findspark.init() and change the cluster_url according.

In [ ]:
import findspark
findspark.init()

In [ ]:
cluster_url = 'local'

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master(cluster_url)\
        .appName("Colab")\
        .config('spark.ui.port', '4040')\
        .getOrCreate()
sc = spark.sparkContext

# Basic Spark Commands

In [ ]:
sc

## Simple RDD Operations
- *sc.parallelize(data)* 
create an RDD from data
- *rdd.count()* 
count number of elements in an rdd
- *rdd.filter(func)* 
create a new rdd from existing rdd and keep only those elements that func is true

In [ ]:
data = [1, 2, 3, 4, 5]
rdd = sc.parallelize(data)
n = rdd.count()
print('count = {0}'.format(n))
l = rdd.collect()
print(l)

In [ ]:
l = rdd.take(3)
print(l)

In [ ]:
f_rdd = rdd.filter(lambda d: d > 2)
for d in f_rdd.collect():
    print(d)
print('filter count = {0}'.format(f_rdd.count()))

## RDD Operations - map and reduce
- *rdd.map(func)*
create a new rdd by performing function func on each element in an rdd
- *rdd.reduce(func)*
aggregate all elements in an rdd using function func

In [ ]:
data = ['line 1', '2', 'more lines', 'last line']

In [ ]:
lines = sc.parallelize(data)

In [ ]:
print(lines.collect())

In [ ]:
lineLengths = lines.map(lambda line: len(line))
print(lineLengths.collect())

In [ ]:
totalLength = lineLengths.reduce(lambda a, b: a+b)
print(totalLength)

In [ ]:
data = (1,2,3,4)
rdd = sc.parallelize(data)
rdd2 = rdd.map(lambda x: x*2)
print(rdd2.collect())
sum_val = rdd2.reduce(lambda a, b: a+b)
print('sum = {0}'.format(sum_val))
mul_val = rdd2.reduce(lambda a, b: a*b)
print('mul = {0}'.format(mul_val))

## RDD Operations - aggregate

In [ ]:
rdd.aggregate((0, 0),
              lambda acc, value: (acc[0]+value, acc[1]+1), 
              lambda acc1, acc2: (acc1[0]+acc2[0], acc1[1]+acc2[1]))

In [ ]:
lines.aggregate(("", 0),
                lambda a, s: (a[0]+s, a[1]+len(s)),
                lambda a, b: (a[0]+b[0], a[1]+b[1]))

# Working with Text

Before running this example, make sure that a data file 'star-wars.txt' has been uploaded to content folder of this colab

In [ ]:
sw = sc.textFile('star-wars.txt')
print('Total = {0} lines'.format(sw.count()))
for line in sw.take(10):
    print('{0}: [{1}]'.format(len(line), line))

In [ ]:
nb_lines = sw.filter(lambda line: len(line) > 0)
print('Non blank line = {0} lines'.format(nb_lines.count()))
all_lowers = nb_lines.map(lambda line: line.lower())
for line in all_lowers.take(10):
    print('{0}: [{1}]'.format(len(line), line))

In [ ]:
words = all_lowers.flatMap(lambda line: line.split())
for w in words.take(5):
    print(w)

In [ ]:
mappers = words.map(lambda word: (word, 1))
counts = mappers.reduceByKey(lambda x, y: x+y)
for wc in counts.take(10):
    print(wc)